# 데이터 불러오기

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

df = pd.read_csv('../datasets/빅분기 실기 데이터/Fvote.csv', encoding='utf-8')
df.head()

,gender_female,gender_male,region_Chungcheung,region_Honam,region_Others,region_Sudo,region_Youngnam,edu,income,age,score_gov,score_progress,score_intention,vote,parties
0,0,1,0,0,0,0,1,3,3,3,2,2,4.0,1,2
1,0,1,0,0,1,0,0,2,3,3,2,4,3.0,0,3
2,0,1,0,1,0,0,0,1,2,4,1,3,2.8,1,4
3,1,0,0,0,0,1,0,2,1,3,5,4,2.6,1,1
4,0,1,0,0,0,1,0,1,2,4,4,3,2.4,1,1


# train / test 데이터 나누기

In [3]:
X = df.drop('vote', axis=1)
y = df[['vote']]

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11, stratify=y)

# 모델 적용

In [5]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()

# 교차검증

## cross_val_score
- 데이터셋의 순사가 있는 그대로 나눠짐
- 즉, 1~30번, 31~60번, 61~90번 등으로 -> 편향 문제 발생 가능성 있음

In [6]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=5)
print('5개 테스트 셋 정확도:', scores)
print('정확도 평균:', scores.mean())

5개 테스트 셋 정확도: [0.73529412 0.64705882 0.73529412 0.75757576 0.66666667]
정확도 평균: 0.7083778966131906


## 랜덤 있는 교차검증 k-fold
- cross_val_score 보완

In [7]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=42)
score = cross_val_score(model, X_train, y_train, cv=kfold)
print('5개 폴드의 정확도:', score)

5개 폴드의 정확도: [0.67647059 0.70588235 0.73529412 0.72727273 0.6969697 ]


## 임의분할 교차검증
- 훈련 데이터와 테스트 데이터를 구성할 때 다른 교차검증에 사용되었던 데이터도 랜덤으로 선택되게 하는 방법
- 전체 데이터 중 일부는 훈련 데이터 또는 테스트 데이터 어디에서도 선택되지 않을 수 있음

In [10]:
from sklearn.model_selection import ShuffleSplit

shuffle_split = ShuffleSplit(n_splits=5, test_size=0.5, train_size=0.5, random_state=11)
score = cross_val_score(model, X_train, y_train, cv=shuffle_split)
print('교차검증 정확도:', score)

교차검증 정확도: [0.66666667 0.73809524 0.6547619  0.78571429 0.71428571]


# train-validity-test 분할과 교차검증

In [11]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=11)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_val, y_train_val, random_state=3)

In [12]:
model.fit(X_train, y_train)
scores = cross_val_score(model, X_train, y_train, cv=5)
print('교차검증 정확도:', score)
print('정확도 평균:', scores.mean())

교차검증 정확도: [0.66666667 0.73809524 0.6547619  0.78571429 0.71428571]
정확도 평균: 0.719927536231884


In [13]:
model.score(X_valid, y_valid)

0.6

In [14]:
model.score(X_test, y_test)

0.7735849056603774